In [1]:
import pandas as pd, numpy as np
import tldextract,re

In [2]:
df = pd.read_excel('USA_SAAS_CB_10Dec16.xlsx')
df = df.dropna(subset=['category_list'])
df.index = range(len(df))
df1 = pd.read_csv('USA_SAAS_CB_10Dec16_website_text.csv')
df1['website_text'] = df1['website_text'].fillna('').apply(lambda x: re.sub('\n',' ',x.lower()))
df['domain_cleaned'] = df['homepage_url'].apply(lambda x: tldextract.extract(x.lower()).domain)
df1['domain_cleaned'] = df1['website'].apply(lambda x: tldextract.extract(x.lower()).domain)
df = df.drop_duplicates('domain_cleaned')
df1 = df1.drop_duplicates('domain_cleaned')
df_final = pd.merge(df,df1,on='domain_cleaned')
df.shape,df1.shape,df_final.shape

No handlers could be found for logger "tldextract"


((2391, 21), (1906, 3), (1893, 23))

In [3]:
df_final.tail()

,li_cpy_name,country_code,state_code,region,city,category_list,category_group_list,industry,short_description,description,...,funding_rounds,funding_total_usd,last_funding_on,homepage_url,linkedin_url,domain,email,domain_cleaned,website,website_text
1888,NuORDER,USA,CA,Los Angeles,West Hollywood,b2b|e-commerce|fashion|internet|saas|wholesale,commerce and shopping|design|internet services,Wholesale,NuORDER is a cloud & mobile B2B eCommerce plat...,NuORDER empowers B2B eCommerce sites for over ...,...,3,13900000.0,2015-02-01 00:00:00,http://www.nuorder.com,http://www.linkedin.com/company/2779405,nuorder.com,info@nuorder.com,nuorder,http://www.nuorder.com,nuorder #1 b2b ecommerce solution brands retai...
1889,EvoNexus (CommNexus),USA,CA,San Diego,La Jolla,big data|cyber security|hardware|life science|...,biotechnology|data and analytics|hardware|info...,Wireless,EvoNexus is a non-profit technology incubator ...,"EvoNexus (formerly CommNexus, formerly the Tel...",...,0,NaN,NaN,http://www.evonexus.org,http://www.linkedin.com/company/commnexus,evonexus.org,NaN,evonexus,http://www.evonexus.org,evonexus jobs contact us resources about about...
1890,Macheen Inc,USA,TX,Austin,Austin,enterprise software|internet|saas,internet services|software,Wireless,Macheen is a SaaS-based platform that brings m...,"Macheen, Inc is a SaaS company that brings mob...",...,5,34394995.0,2013-12-01 00:00:00,http://macheen.com,https://www.linkedin.com/company/macheen-inc,macheen.com,info@macheen.com,macheen,http://macheen.com,www.macheen.com
1891,RTP Holdings,USA,PA,Philadelphia,Radnor,saas,NaN,Wireless,"Indoor location services, strategy, solutions....","Indoor GPS makes everyday activities faster, e...",...,0,NaN,NaN,http://www.rtpholdings.com,http://www.linkedin.com/company/rtp-holdings,rtpholdings.com,NaN,rtpholdings,http://www.rtpholdings.com,rtp holdings - rtp holdings home solution bene...
1892,SeatServe,USA,NY,New York City,New York,e-commerce|mobile|sports,commerce and shopping|mobile|sports,Wireless,A real-time delivery SaaS solution designed fo...,SeatServe is a digital solutions provider base...,...,0,NaN,NaN,http://seatserve.com,https://www.linkedin.com/company/seatserve,seatserve.com,info@seatserve.com,seatserve,http://seatserve.com,seatserve - stay seated. in-seat delivery at y...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [40]:
def category_tokenizer(text):
    ''' split by pipe (|) and return'''
    text = text.strip()
    text = re.sub('\|$|^\|','',text)
    if text == '|':
        return []
    return [i.strip() for i in text.lower().split('|')]
vectorizer = CountVectorizer(max_df=1.0, min_df=50,tokenizer=category_tokenizer,binary=True)
dv_matrix = vectorizer.fit_transform(df_final['category_list'].fillna('')+'|'+df_final['category_group_list'].fillna(''))
dv_matrix.shape

(1893, 38)

In [41]:
np.random.seed(5)
msk1 = np.random.rand(dv_matrix.shape[0]*dv_matrix.shape[1]).reshape(dv_matrix.shape) < 0.5
dv_matrix[msk1] = -1

In [5]:
tfidf_vectorizer_descr = TfidfVectorizer(max_df=0.8, min_df=0.01,stop_words='english')
X_descr_train = tfidf_vectorizer_descr.fit_transform(df_final['description'].fillna(''))

tfidf_vectorizer_website_text = TfidfVectorizer(max_df=0.8, min_df=0.01,stop_words='english')
X_website_text_train = tfidf_vectorizer_website_text.fit_transform(df_final['website_text'].fillna(''))
dv_matrix.shape,X_descr_train.shape,X_website_text_train.shape

/usr/local/lib/python2.7/dist-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


((1893, 38), (1893, 1108), (1893, 5323))

In [6]:
pd.Series(dv_matrix.toarray()[:,3]).value_counts()

-1    961
 0    894
 1     38
dtype: int64

In [8]:
import cotraining_multi
reload(cotraining_multi)
print 'naive bayes multi'
clf = cotraining_multi.CotrainMulti() #MultinomialNB(fit_prior=False)
clf.fit(MultinomialNB(fit_prior=False),X_descr_train,X_website_text_train,dv_matrix.toarray())

naive bayes multi
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:28
no of iterations took for fitting:30
no of iterations took for fitting:22
no of iterations took for fitting:30
no of iterations took for fitting:22
no of iterations took for fitting:30
no of iterations took for fitting:20
no of iterations took for fitting:30
no of iterations took for fitting:24
no of iterations took for fitting:30
no of iterations took for fitting:27
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:24
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:30
no of iterations took for fitting:24
no of iterations too

In [9]:
i=0
import pdb
while i<1:
    pdb.set_trace()
    y_pred = clf.predict(X_descr_train,X_website_text_train)
    i+=1

> <ipython-input-9-c1bd7db5e61d>(5)<module>()
-> y_pred = clf.predict(X_descr_train,X_website_text_train)
(Pdb) c


In [10]:
y_pred_prob_pos,y_pred_prob_neg = clf.predict_proba(X_descr_train,X_website_text_train)

/usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((1893, 2), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [ ]:
print pd.Series(y_pred[101,:]).value_counts()
print pd.Series(y_pred_prob_pos[101,:]>=0.5).value_counts()